In [ ]:
import os
import re

The following will determine that you have a sqlite database.

In [ ]:
if "us.sqlite" in os.listdir("data"):
    print "You appear to have the us.sqlite database in the correct location."
else:
    print """You don't appear to have the us.sqlite database.  
Please download https://cdn.radiocells.org/us.sqlite ,
and put the database in the ./data/ directory.
"""

The following cell will determine if you have sqlite3.

In [ ]:
if "sqlite3" in os.listdir("/usr/bin/"):
    print "You have sqlite3."
else:
    print """You don't appear to have sqlite3.  You can download it here: https://www.sqlite.org/download.html
If you think you already have it in a nonstandard location, you can try running the commands below anyway."""

The following will build the localization binary. This will probably print nothing if everything goes well.

In [ ]:
build_command = "g++ localization_main.cc lib/estimation_functions.cc lib/query_functions.cc -o localization -std=c++11 -l sqlite3"
with os.popen(build_command) as build_output:
    print build_output.read()

The following will get get all nearby bssids and corresponding rssis.

On Mac OS:

In [ ]:
def process_raw_lines_from_apple_airport(raw_lines):
    items = list()
    pat = r"([a-z0-9]{2}:){5}[a-z0-9]{2} -[0-9]+"
    for line in raw_lines:
        m = re.search(pat, line)
        if m:
            items.append(m.group(0).upper().replace(":", ""))
    return items

def get_bssids_and_rssis_via_apple_airport():
    airport_command = "/System/Library/PrivateFrameworks/Apple80211.framework/Versions/Current/Resources/airport -s"
    with os.popen(airport_command) as f:
        raw_lines = f.readlines()
    results = process_raw_lines_from_apple_airport(raw_lines)
    return " ".join(results)

bssids_and_rssis = get_bssids_and_rssis_via_apple_airport()

print bssids_and_rssis

On Linux:

In [ ]:
WIFI_INTERFACE_NAME = 'wlan0'

def process_raw_lines_from_linux_iw(raw_lines):
    items = list()
    bss_pat = r"([a-z0-9]{2}:){5}[a-z0-9]{2}"
    rssi_pat = r"signal: (-[0-9]+.[0-9]+) dBm"
    
    start_indices = [i for i in range(len(raw_lines)) if re.search(bss_pat, raw_lines[i])]
    for i in range(len(start_indices)):
        start_index = start_indices[i]
        next_start_index = start_indices[i+1] if i+1 < len(start_indices) else len(raw_lines)
        bssid = re.search(bss_pat, raw_lines[start_index]).group(0).upper().replace(":", "")
        for j in range(start_index, next_start_index):
            m = re.search(rssi_pat, raw_lines[j])
            if m:
                rssi = m.group(1)
                break
        items.append(bssid + " " + rssi)
    return items


def get_bssids_and_rssis_via_linux_iw():
    iw_command = "sudo iw " + WIFI_INTERFACE_NAME + " scan"
    with os.popen(iw_command) as f:
        raw_lines = f.readlines()
    result_list = process_raw_lines_from_linux_iw(raw_lines)
    return " ".join(result_list)

bssids_and_rssis = get_bssids_and_rssis_via_linux_iw()

print bssids_and_rssis

The following command will run the localization program with the preceeding bssids/rssis and print the output.  If any of your local bssids were found in the database, your coordinate estimate will be printed at the bottom.

In [ ]:
localize_command = "./localization " + bssids_and_rssis
with os.popen(localize_command) as g:
    print g.read()